# Calculating tide statistics and satellite biases

**This guide demonstrates how to use the [`tide_stats`](../../api/#eo_tides.stats.tide_stats) and  [`pixel_stats`](../../api/#eo_tides.stats.pixel_stats) functions from [`eo_tides.stats`](../../api/#eo_tides.stats) to calculate local tide statistics and identify biases caused by interactions between tidal processes and satellite orbits.**

Complex interactions between temporal tide dynamics and the regular mid-morning overpass timing of sun-synchronous sensors like Landsat or Sentinel-2 mean that satellites often does not observe the entire tidal cycle. 
Biases in satellite coverage of the tidal cycle can mean that tidal extremes (e.g. the lowest or highest tides at a location) may either never be captured by satellites, or be over-represented in the satellite EO record. 
Local tide dynamics can cause these biases to vary greatly both through time and spatially (Figure 1), making it challenging to consistently analyse and compare coastal processes consistently - particularly for large-scale (e.g. regional or global) analyses.

To ensure that coastal EO analyses are not inadvertently affected by tide biases, it is important to compare how well the tides observed by satellites match the full range of tides at a location.
The `tidal_stats` and `pixel_stats` functions compares the subset of tides observed by satellite data against the full range of tides modelled at a regular interval through time (every two hours by default) across the entire time period covered by the satellite dataset.
This comparison is used to calculate several useful statistics and plots that summarise how well your satellite data capture real-world tidal conditions.

> **Figure 1:** Example of tide biases in Landsat satellite data across coastal Australia ([Bishop-Taylor et al. 2018](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3)). "Spread" represents the proportion of the astronomical tide range observed by satellites; low and high tide "offsets" represent the proportion of highest and lowest tides never observed.

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0272771418308783-gr8_lrg.jpg" alt="Tide bias figure">

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        For a more detailed discussion of the issue of tidal bias in sun-synchronous satellite observations of the coastline, refer to the 'Limitations and future work' section in <a href="https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3">Bishop-Taylor et al. 2018.</a>
    </p>
</div>

## Getting started
As in the previous examples, our first step is to tell `eo-tides` the location of our tide model directory (if you haven't set this up, [refer to the setup instructions here](../../setup)):

In [ ]:
directory = "../../tests/data/tide_models/"

## Load Sentinel-2 satellite data using odc-stac

We can now load a time-series of satellite data over our area of interest using the Open Data Cube's `odc-stac` package. 
In this example, we will load **Sentinel-2** satellite data from **2021-2023** over the city of **Broome, Western Australia**.
We will load this data from the [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) STAC catalogue.

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        For a more detailed guide to using STAC metadata and <code>odc-stac</code> to find and load satellite data, refer to the <a href="https://knowledge.dea.ga.gov.au/guides/setup/gis/stac/">STAC user guide</a> here.
    </p>
</div>

In [ ]:
import odc.stac
import pystac_client
import planetary_computer

# Connect to STAC catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Set cloud access defaults
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

# Build a query and search the STAC catalog for all matching items
bbox = [122.160, -18.05, 122.260, -17.95]
query = catalog.search(
    bbox=bbox,
    collections=["sentinel-2-l2a"],
    datetime="2021/2023",
)

# Load data into xarray format
ds_s2 = odc.stac.load(
    items=list(query.items()),
    bands=["red"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
    fail_on_error=False,
    chunks={},
)

print(ds_s2)

## Using tide_stats

Once we have loaded some satellite data, we can pass this to the `tide_stats` function to calculate local tide statistics and reveal any potential tidal biases.
The `tide_stats` function will return a plain-text summary below, as well as a visual plot that compares the distribution of satellite-observed tides (black dots) against the full range of modelled astronomical tide conditions (blue) using three useful metrics:

1. **Spread:** The proportion of the full modelled astronomical tidal range that was observed by satellites. A high value indicating good coverage of the tide range.
2. **Offset high:** The proportion of the highest tides not observed by satellites at any time, as a proportion of the full modelled astronomical tidal range. A high value indicates that the satellite data is biased towards never capturing high tides.
3. **Offset low:** The proportion of the lowest tides not observed by satellites at any time, as a proportion of the full modelled astronomical tidal range. A high value indicates that the satellite data is biased towards never capturing low tides.

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        For a more detailed description of these biases, see <a href="https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3">Bishop-Taylor et al. 2018.</a>
    </p>
</div>

In [ ]:
from eo_tides.stats import tide_stats

statistics_df = tide_stats(
    data=ds_s2,
    directory=directory,
)


As we can see in the graph, Sentinel-2 captured a biased proportion of the tide range at this location: only observing ~64% of the tide range, and never observing the lowest 26% of tides.

The `tide_stats` function also outputs a `pandas.Series` object containing statistics for the results above, including:

* `y`: latitude used for modelling tide heights
* `x`: longitude used for modelling tide heights
* `mot`: mean tide height observed by the satellite (metres)
* `mat`: mean modelled astronomical tide height (metres)
* `lot`: minimum tide height observed by the satellite (metres)
* `lat`: minimum tide height from modelled astronomical tidal range (metres)
* `hot`: maximum tide height observed by the satellite (metres)
* `hat`: maximum tide height from modelled astronomical tidal range (metres)
* `otr`: tidal range observed by the satellite (metres)
* `tr`: modelled astronomical tide range (metres)
* `spread`: proportion of the full modelled tidal range observed by the satellite
* `offset_low`: proportion of the lowest tides never observed by the satellite
* `offset_high`: proportion of the highest tides never observed by the satellite

In [ ]:
statistics_df

### Compare against Sentinel-1 tide biases

In the previous example, we saw that Sentinel-2 data was biased towards never capturing low tide at our location.
These biases are caused by the consistent 10:30 am local overpass time of the Sentinel-2 satellites, which due to a phenomenon called "tide-aliasing" means that certain tides never occur when the satellite overpasses.

One possible way around these biases is to use different satellite data from satellites that overpass at different times.
For example, Sentinel-1 radar satellites follow a different orbit to Sentinel-2, overpassing at a local time of 6:00 pm instead of 10:30 am. 
This diference in overpass time potentially means that Sentinel-1 satellite data may capture different tides to Sentinel-2.

In the next example, we run the `tide_stats` function on data loaded from **Sentinel-1** for the same location and time period to see if this data is affected by the same tide biases.

In [ ]:
# Build a query and search the STAC catalog for all matching items
bbox = [122.160, -18.05, 122.260, -17.95]
query = catalog.search(
    bbox=bbox,
    collections=["sentinel-1-rtc"],
    datetime="2021/2023",
)

# Load data into xarray format
ds_s1 = odc.stac.load(
    items=list(query.items()),
    bands=["vv"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
    fail_on_error=False,
    chunks={},
)

print(ds_s1)

When we run `tide_stats`, we can see a very different pattern: Sentinel-1 data is biased towards *low tide* observations, and never observes high tide at our location!

In [ ]:
statistics_df = tide_stats(
    data=ds_s1,
    directory=directory,
)


### Calculate biases for multiple satellite sensors
At our location, Sentinel-2 optical satellites are biased towards high tide observations, while Sentinel-1 radar satellites are biased towards low tide observations. 
Could combining data from multiple EO sensors help us capture a more complete view of tides at this location?

To test this theory, we can combine Sentinel-2 and Sentinel-1 data into a single `xarray.Dataset`, recording the name of each sensor using a new `satellite_name` coordinate in our data:

In [ ]:
import xarray as xr

# Give each observation a "satellite_name" based on its satellite
ds_s1 = ds_s1.assign_coords(satellite_name=("time", ["Sentinel-1"] * ds_s1.sizes["time"]))
ds_s2 = ds_s2.assign_coords(satellite_name=("time", ["Sentinel-2"] * ds_s2.sizes["time"]))

# Combine both Sentinel-1 and Sentinel-2 data into a single dataset
ds_all = xr.concat([ds_s1, ds_s2], dim="time")
print(ds_all)

We can now run `tide_stats` again.
This time, we pass our satellite name coordinate to the function using the `plot_col="satellite_name"` parameter.
This will plot data from each of our satellites using a different symbol.

In [ ]:
statistics_df = tide_stats(
    data=ds_all,
    plot_col="satellite_name",
    directory=directory,
)


We can see that at this location, combining Sentinel-2 and Sentinel-1 data greatly improves our biases: our satellite data now covers ~85% of the modelled astronomical tide range, and only fails to observe 10% of the highest tides and 6% of the lowest tides!

## Using pixel_stats

### Modelling tide statistics and biases spatially

Because tide regimes and satellite biases can vary greatly along the coast, it can be useful to plot these biases spatially.
To do this, we can use the `pixel_stats` function. 

`pixel_stats` works similarly to `tide_stats`, except statistics are calculated across the entire extent of your satellite dataset. 
The function will generate an `xarray.Dataset` output containing the statistics discussed above as two-dimensional arrays.

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        The <code>pixel_stats</code> function uses <a href="../../api/#eo_tides.eo.pixel_tides"><code>eo_tides.eo.pixel_tides</code></a> to model tides spatially. You can experiment passing in parameters like <code>resolution</code> and <code>buffer</code> to customise the modelling grid used for calculating tide biases. Be warned however that you can quickly run out of memory with large analyses, given the number of timesteps required to model astronomical low and high tide.
    </p>
</div>

In [ ]:
from eo_tides.stats import pixel_stats

stats_ds = pixel_stats(
    data=ds_s2,
    directory=directory,
)
print(stats_ds)

We can explore these statistics on a map:

In [ ]:
stats_ds.spread.odc.explore()

Or plot them directly:

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
stats_ds.spread.plot(ax=axes[0], vmin=0.5, vmax=0.9, cmap="magma_r", add_labels=False)
stats_ds.offset_low.plot(ax=axes[1], vmin=0, vmax=0.35, cmap="magma", add_labels=False)
stats_ds.offset_high.plot(ax=axes[2], vmin=0, vmax=0.35, cmap="magma", add_labels=False)
axes[0].set_title("Spread")
axes[1].set_title("Low tide offset")
axes[2].set_title("High tide offset");

## Next steps

We have explored calculating tide statistics and biases in EO data. Now we can learn how to validate modelled tides against [measured tide gauge data](../Validating_tides) to ensure the tides we are modelling are accurate.